In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [3]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

D0710 04:27:43.077882991      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0710 04:27:43.077927881      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0710 04:27:43.077932212      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0710 04:27:43.077935635      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0710 04:27:43.077938735      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0710 04:27:43.077941966      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0710 04:27:43.077945135      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0710 04:27:43.

In [4]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.2 MB/s eta 0:00:00


In [6]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


In [7]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

In [8]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:0000:010:01


**xlm roberta large**

In [9]:
from transformers import RobertaConfig, RobertaModel
from transformers import TFXLMRobertaModel, XLMRobertaTokenizer
from transformers import XLMRobertaConfig, XLMRobertaModel

# Initializing an XLM-RoBERTa configuration
configuration = XLMRobertaConfig()

# Initializing a model (with random weights) from the configuration
model = XLMRobertaModel(configuration)

# Accessing the model configuration
configuration = model.config

import tensorflow as tf

model_name = 'xlm-roberta-large'
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('</s>') #sep
    return tokenizer.convert_tokens_to_ids(tokens)

def roberta_encode(hypotheses, premises, tokenizer):
    num_examples = len(hypotheses)
  
    sentence1 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(hypotheses)])

    sentence2 = tf.ragged.constant([
        encode_sentence(s)
        for s in np.array(premises)])
    
    cls = [tokenizer.convert_tokens_to_ids(['[s]'])]*sentence1.shape[0]
    
    input_word_ids = tf.concat([cls,sentence1,sentence2],axis=-1)
    
    input_mask = tf.ones_like(input_word_ids).to_tensor(shape=[input_word_ids.shape[0], 128])
    
    type_cls = tf.zeros_like(cls)
    
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat(
        [type_cls, type_s1, type_s2], axis=-1).to_tensor(shape=[input_word_ids.shape[0], 128])

    inputs = {
        'input_word_ids': input_word_ids.to_tensor(shape=[input_word_ids.shape[0], 128]),
        'input_mask': input_mask,
        'input_type_ids': input_type_ids}

    return inputs

train_input = roberta_encode(train.premise.values, train.hypothesis.values, tokenizer)

max_len = 128

*** layer 3개 + 기본 데이터셋 ***

In [10]:
def build_model():
    roberta_encoder = TFXLMRobertaModel.from_pretrained(model_name)
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = roberta_encoder([input_word_ids, input_mask, input_type_ids])[0]
    x = tf.keras.layers.Dense(128, activation='relu')(embedding[:, 0, :])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output = tf.keras.layers.Dense(3, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

with strategy.scope():
    model = build_model()
    model.summary()
    
model.fit(train_input, train.label.values, epochs = 18, verbose = 1, batch_size = 256, validation_split = 0.25)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tfxlm_roberta_model (TFXLMRobe  TFBaseModelOutputWi  559890432  ['input_word_ids[0][0]',         
 rtaModel)                      thPoolingAndCrossAt               'input_mask[0][0]',         

2023-07-10 04:31:59.178514: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-10 04:32:01.588354: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


36/36 [==============================] - ETA: 0s - loss: 1.3749 - accuracy: 0.3331

2023-07-10 04:35:27.938369: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-10 04:35:28.511339: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


36/36 [==============================] - 309s 4s/step - loss: 1.3749 - accuracy: 0.3331 - val_loss: 1.1435 - val_accuracy: 0.3432
Epoch 2/18
36/36 [==============================] - 16s 441ms/step - loss: 1.2974 - accuracy: 0.3462 - val_loss: 1.1607 - val_accuracy: 0.3432
Epoch 3/18
36/36 [==============================] - 16s 441ms/step - loss: 1.2863 - accuracy: 0.3494 - val_loss: 1.1604 - val_accuracy: 0.3432
Epoch 4/18
36/36 [==============================] - 16s 443ms/step - loss: 1.2419 - accuracy: 0.3625 - val_loss: 1.1110 - val_accuracy: 0.3455
Epoch 5/18
36/36 [==============================] - 16s 445ms/step - loss: 1.1084 - accuracy: 0.4629 - val_loss: 1.0121 - val_accuracy: 0.4393
Epoch 6/18
36/36 [==============================] - 16s 444ms/step - loss: 0.8958 - accuracy: 0.6055 - val_loss: 0.8167 - val_accuracy: 0.6611
Epoch 7/18
36/36 [==============================] - 16s 444ms/step - loss: 1.0172 - accuracy: 0.5294 - val_loss: 0.8220 - val_accuracy: 0.6347
Epoch 8/18
3

In [11]:
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [12]:
test_input = roberta_encode(test.premise.values, test.hypothesis.values, tokenizer)

In [13]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

2023-07-10 04:40:49.516684: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-10 04:40:50.068823: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


163/163 [==============================] - 35s 88ms/step


In [14]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions

In [15]:
submission.head()

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1


In [16]:
submission.to_csv("submission.csv", index = False)

In [17]:
print(submission)

              id  prediction
0     c6d58c3f69           2
1     cefcc82292           1
2     e98005252c           0
3     58518c10ba           1
4     c32b0d16df           1
...          ...         ...
5190  5f90dd59b0           0
5191  f357a04e86           2
5192  1f0ea92118           0
5193  0407b48afb           0
5194  16c2f2ab89           2

[5195 rows x 2 columns]
